In [1]:
import yfinance as yf

# Download historical data for a specific stock
ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end='2023-01-01')

[*********************100%***********************]  1 of 1 completed


In [2]:
# Check for missing values
print(data.isnull().sum())

# Fill missing values using forward fill
data.fillna(method='ffill', inplace=True)

Price   Ticker
Close   AAPL      0
High    AAPL      0
Low     AAPL      0
Open    AAPL      0
Volume  AAPL      0
dtype: int64


C:\Users\KIIT\AppData\Local\Temp\ipykernel_12892\2600411678.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [3]:
from sklearn.preprocessing import MinMaxScaler

# Scale the 'Close' price
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

In [6]:
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Prepare data for linear regression
X = np.arange(len(data)).reshape(-1, 1)
y = data['Close'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
lr_predictions = lr_model.predict(X_test)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Prepare data for LSTM
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        X.append(a)
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape input to be [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
lstm_model.add(LSTM(units=50, return_sequences=False))
lstm_model.add(Dense(units=1))

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Make predictions
lstm_predictions = lstm_model.predict(X_test)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

Epoch 1/10


c:\python\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 0.0044
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 8.7421e-05
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 6.8107e-05
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 5.9854e-05
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 5.5010e-05
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - loss: 5.8292e-05
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - loss: 5.6258e-05
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - loss: 4.6890e-05
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - loss: 4.7771e-05
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - loss: 4.3613e-05
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step


In [11]:
from sklearn.metrics import mean_squared_error

# Calculate RMSE for Linear Regression
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_predictions))
print(f'Linear Regression RMSE: {lr_rmse}')

# Calculate RMSE for LSTM
lstm_rmse = np.sqrt(mean_squared_error(scaler.inverse_transform(y_test.reshape(-1, 1)), lstm_predictions))
print(f'LSTM RMSE: {lstm_rmse}')

ValueError: Found input variables with inconsistent numbers of samples: [594, 655]